Query provided by Amazon:


```sql
create table event(
	eventid integer not null distkey,
	venueid smallint not null,
	catid smallint not null,
	dateid smallint not null sortkey,
	eventname varchar(200),
	starttime timestamp);
```

In [ ]:
import sqlite3
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
data = pd.read_csv('tickitdb/allevents_pipe.txt', sep="|", header=None)
display(data.head())
data.columns = ['eventid', 'venueid', 'catid', 'dateid', 'eventname', 'starttime']
display(data.head())

In [ ]:
# Make sure event id is actually a primary key
assert len(data.eventid) == len(data.eventid.unique())

In [ ]:
con = sqlite3.connect('tickit.db')
cursor = con.cursor()
print("Successfully Connected to Database")

# Drop table if already exists
cursor.execute("DROP TABLE IF EXISTS event")
con.commit()

# Create Table
sql_create_table = """CREATE TABLE IF NOT EXISTS event (
                                eventid integer not null,
                                venueid smallint not null,
                                catid smallint not null,
                                dateid smallint not null,
                                eventname varchar(200),
                                starttime timestamp
                                );"""
cursor.execute(sql_create_table)
con.commit()
print("Succesfully create event table")

# Insert rows from extracted entity dataframe
for i, row in tqdm(data.iterrows(), 
                   total=len(data)):
    sqlite_insert_query = f"""INSERT INTO event
                          (eventid, venueid, catid, dateid, eventname, starttime) 
                           VALUES 
                          ('{row['eventid']}', '{row['venueid']}',  '{row['catid']}', 
                          '{row['dateid']}', '{row['eventname'].replace("'", "''")}', '{row['starttime']}')"""

    cursor.execute(sqlite_insert_query)
    con.commit()
print(f"Successfully inserted {len(data)} rows into the event table")
cursor.close()